In [3]:
import numpy as np
from tqdm import tqdm_notebook
import itertools

In [4]:
N = 90
k = N-1
covmat = np.random.rand(N,N)
covmat[[range(0,N)],[range(0,N)]] = N
arr = list(range(0,N))
#covmat

In [5]:
#arr

In [6]:
covmat = 0.5*(covmat+covmat)

In [5]:
from ipyparallel import Client
rc = Client(profile='pucv-cluster')
rc.purge_everything()

In [6]:
rc

In [7]:
rc.ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58]

In [8]:
dview = rc[:] # use all engines

In [9]:
dview['covmat'] = covmat

In [10]:
dview.block = True
#dview.pull('covmat', targets=0)

In [11]:
"""N//"""

'N//'

In [7]:
sub_mat_n = N//2
hola = itertools.combinations(arr, sub_mat_n)

In [8]:
from math import factorial
n_el = N

n = factorial(n_el) / factorial(sub_mat_n) / factorial(n_el-sub_mat_n)
n_per_process = int(n)//59#len(rc.ids) 

print(n,n_per_process)

1.0382742128755342e+26 1759786801483956263685935


In [14]:
from itertools import islice, chain

def batch(iterable, size):
    sourceiter = iter(iterable)
    while True:
        batchiter = islice(sourceiter, size)
        yield chain([next(batchiter)], batchiter)

In [15]:
indexes = []
for i in range(len(rc.ids)):
    batchiter = islice(hola, int(round(n/len(rc.ids),0)))
    ind = chain([next(batchiter)], batchiter)
    indexes.append(ind)
#for el in hola2:
#    print(el)

In [16]:
indexes

In [17]:
dview.scatter('index_mat',indexes)

In [18]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


In [19]:
def det(N):
    dets = numpy.zeros(N,dtype=numpy.float32)
    #dets = []
    idx_mat = index_mat[0]
    i=0
    for ind in idx_mat:
        idx = list(ind)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        i+=1
        #dets.append(numpy.linalg.det(matrix))
    return dets

In [20]:
ar = dview.apply_async(det,int(round(n/len(rc.ids),0)))
ar.wait_interactive()

  59/59 tasks finished after   75 s
done


In [21]:
res = ar.get()

In [22]:
res[0]

array([1.3966487e+22, 1.3945454e+22, 1.3927510e+22, ..., 1.3985826e+22,
       1.3993240e+22, 1.4001013e+22], dtype=float32)

In [23]:
aux = 0
for el in res:
    aux += len(el)
    #aux += len(el)
aux

155117549

In [24]:
round(n/59,0)

2629111.0

In [28]:
import numpy as np

dets_iterative = np.zeros(int(n))
hola = itertools.combinations(arr, sub_mat_n)


In [29]:
%%time
i = 0
for ind in tqdm_notebook(hola):
    idx = list(ind)
    matrix = covmat[idx][:,idx]
    dets_iterative[i] = numpy.linalg.det(matrix)
    i+=1


CPU times: user 38min 19s, sys: 17.4 s, total: 38min 37s
Wall time: 38min 7s


In [34]:
dets_iterative[0] 

1.3966486414803868e+22

In [33]:
res[0]

array([1.3966487e+22, 1.3945454e+22, 1.3927510e+22, ..., 1.3985826e+22,
       1.3993240e+22, 1.4001013e+22], dtype=float32)

In [35]:
ar.metadata[0]

{'msg_id': 'eccaa0de-3723b55a0a6dc8bc491f2763',
 'submitted': datetime.datetime(2019, 6, 7, 0, 56, 58, 556902, tzinfo=tzutc()),
 'started': datetime.datetime(2019, 6, 7, 0, 56, 58, 580221, tzinfo=tzutc()),
 'completed': datetime.datetime(2019, 6, 7, 0, 58, 11, 315985, tzinfo=tzutc()),
 'received': datetime.datetime(2019, 6, 7, 0, 58, 11, 831527, tzinfo=tzutc()),
 'engine_uuid': '375ada9a-214b906d05999294582a4f96',
 'engine_id': 0,
 'follow': [],
 'after': [],
 'status': 'ok',
 'execute_input': None,
 'execute_result': None,
 'error': None,
 'stdout': '',
 'stderr': '',
 'outputs': [],
 'data': {}}

In [47]:
dets = np.zeros(9033567229484759421,dtype=np.float32)

for el in tqdm_notebook(dets):
    pass

ValueError: array is too big; `arr.size * arr.dtype.itemsize` is larger than the maximum possible size.

In [36]:
import pandas as pd

In [39]:
import numpy as np
Cs, epsilons = pd.np.meshgrid(np.logspace(-2,3,20),[i/20 for i in range(1,21)])

len(list(zip(Cs.reshape((-1,1)), epsilons.reshape((-1,1)))))

400